In [2]:
import re
from datasets import load_dataset

DATASET_NAME = "verifiers-for-code/humaneval_plan_generation"
COL_TO_CLEAN = "l3_plans_new"
COL_CLEANED_NAME = "cleaned-" + COL_TO_CLEAN

dataset = load_dataset(DATASET_NAME, split="test")

def extract_plan(text):
    start_match = re.search(r'<plan>', text)
    if start_match:
        start_index = start_match.end()
        end_match = re.search(r'</plan>', text[start_index:])
        
        if end_match:
            end_index = start_index + end_match.start()
            plan = text[start_index:end_index].strip()
        else:
            # If no closing tag, include everything after <plan>
            plan = text[start_index:].strip()
        
        return plan.split('\n')
    else:
        print("No plan found in text")
    return []

def insert_plan_into_docstring(prompt, plan):
    # Find the end of the docstring
    docstring_start = prompt.find('"""')
    docstring_end = prompt.find('"""', docstring_start + 3)
    
    # If there's no docstring or it's malformed, return the original prompt
    if docstring_end == -1:
        return prompt
    
    # Get the indentation of the closing docstring
    closing_indent = prompt.rfind('\n', 0, docstring_end) + 1
    indent = prompt[closing_indent:docstring_end].replace('"""', '')
    
    # Prepare the plan text with proper indentation, without "Plan:" header
    plan_text = "\n\n" + "\n".join(indent + line for line in plan)
    
    return prompt[:docstring_end] + plan_text + prompt[docstring_end:]

# Apply the extraction and insertion to the dataset
def process_item(item):
    plan = extract_plan(item[COL_TO_CLEAN])
    new_prompt = insert_plan_into_docstring(item['prompt'], plan)
    item[COL_CLEANED_NAME] = new_prompt  # Add the new column to the item
    return item  # Return the entire item with the new column

dataset = dataset.map(process_item)

# Print a sample to verify
print(dataset[56][COL_CLEANED_NAME])

Map: 100%|██████████| 164/164 [00:00<00:00, 2617.30 examples/s]



def correct_bracketing(brackets: str):
    """ brackets is a string of "<" and ">".
    return True if every opening bracket has a corresponding closing bracket.

    >>> correct_bracketing("<")
    False
    >>> correct_bracketing("<>")
    True
    >>> correct_bracketing("<<><>>")
    True
    >>> correct_bracketing("><<>")
    False
    

    Action Plan:
    1. Initialize an empty stack to store opening brackets encountered.
    2. Iterate through each character in the input string:
        a. If the character is an opening bracket, push it onto the stack.
        b. If the character is a closing bracket:
            i. Check if the stack is empty (no corresponding opening bracket).
            ii. If the stack is empty, return False (no matching opening bracket).
            iii. Pop the topmost opening bracket from the stack.
    3. After iterating through the entire string, check if the stack is empty:
        a. If the stack is empty, return True (all opening brackets have co

In [5]:
print(dataset[-19][COL_CLEANED_NAME])


def order_by_points(nums):
    """
    Write a function which sorts the given list of integers
    in ascending order according to the sum of their digits.
    Note: if there are several items with similar sum of their digits,
    order them based on their index in original list.

    For example:
    >>> order_by_points([1, 11, -1, -11, -12]) == [-1, -11, 1, -12, 11]
    >>> order_by_points([]) == []
    

    Action Plan:
    1. Define a helper function `sum_of_digits` that takes an integer as input and returns the sum of its digits.
        - Convert the number to a string to iterate over its digits
        - Initialize a variable to store the sum of digits
        - Iterate over each digit in the string:
            - Convert the digit back to an integer
            - Add the integer to the sum
        - Return the sum of digits
    
    2. Use the `sorted` function to sort the input list `nums`.
        - Pass the `sum_of_digits` function as the `key` parameter to the `sorted` fu

In [2]:
dataset.column_names

['task_id',
 'prompt',
 'canonical_solution',
 'test',
 'entry_point',
 'sonnet-3.5_gold_plans',
 'cleaned_sonnet-3.5_gold_plans',
 'generated_phi3_baseline',
 'generated_phi3_plan_generation',
 'phi3-planner-plans',
 'cleaned-phi3-planner-plans',
 'self_planning_Phi-3-mini-4k-instruct',
 'cleaned-self_planning_Phi-3-mini-4k-instruct',
 'phi3-planner-granular',
 'cleaned-phi3-planner-granular',
 'l3_plans_non_gran',
 'cleaned-l3_plans_non_gran']

In [3]:
dataset.push_to_hub(DATASET_NAME, split="test")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  2.39it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/verifiers-for-code/humaneval_plan_generation/commit/52b0482dc76bd8ea2fe262b0bcafb7da3f77bf95', commit_message='Upload dataset', commit_description='', oid='52b0482dc76bd8ea2fe262b0bcafb7da3f77bf95', pr_url=None, pr_revision=None, pr_num=None)